# Homework 7 

Michael Klein


In [2]:
# import necessary modules
# numpy provides powerful multi-dimensional arrays to hold and manipulate data
import numpy as np
# matplotlib provides powerful functions for plotting figures
import matplotlib.pyplot as plt
# astropy provides unit system and constants for astronomical calculations
import astropy.units as u
import astropy.constants as const
# import Latex module so we can display the results with symbols
from IPython.display import Latex
%matplotlib inline

# **** import CenterOfMass to determine the COM pos/vel of M33
from CenterofMass2 import CenterOfMass

# **** import the GalaxyMass to determine the mass of M31 for each component
from GalaxyMass import ComponentMass

# M33AnalyticOrbit

In [11]:
class M33AnalyticOrbit:
    
    """ Calculate the analytical orbit of M33 around M31 """
    def __init__(self, filename, ptype=2):
        # Initializing the instance of this class with the following properties:
        # ptype = 2 refers to disk particles
        # Storing the output file name
        self.filename = filename
        # The gravitational constant
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        # An instance of the CenterOfMass class for M33
        COM_M33 = CenterOfMass("M33_000.txt",2)
        # COM position for M33
        # 0.1 refers to delta being equal to 0.1 and 4 refers to VolDec being equal to 4
        COMP_M33 = (COM_M33.COM_P(0.1,4)).value
        # COM velocity for M33
        COMV_M33 = (COM_M33.COM_V(COMP_M33[0],COMP_M33[1],COMP_M33[2])).value
        # An instance of the CenterOfMass class for M31
        COM_M31 = CenterOfMass("M31_000.txt",2)
        # COM position for M31
        # delta = 0.1 and VolDec = 2
        COMP_M31 = (COM_M31.COM_P(0.1,2)).value
        # COM velocity for M31
        COMV_M31 = (COM_M31.COM_V(COMP_M31[0],COMP_M31[1],COMP_M31[2]))
        # Storing the difference between the position and velocity vectors
        self.r0 = COMP_M33.value - COMP_M31.value
        self.v0 = COMV_M33.value - COMV_M31.value
        
        # Storing the scale lengths and masser for each component of M31
        # The disk radius
        self.rdisk = 5
        # The disk mass in Msun
        self.Mdisk = ComponentMass("M31_000.txt",2)*1e12
        # The bulge radius
        self.rbulge = 1
        # The bulge mass
        self.Mbulge = ComponentMass("M31_000.txt",3)*1e12
        # The halo radius
        self.rhalo = 63
        # The halo mass
        self.Mhalo = ComponentMass("M31_000.txt",1)*1e12
    
    # Defining a function that will take into account the halo and bulge acceleration using Hernquist Profile
    def HernquistAccel(self, M, ra, r):
        """Inputs:
        M = mass of either bulge or halo
        ra = scale length
        r = relative position vector
        Returns:
        Hernquist Acceleration"""
        # Magnitude of the position vector
        rmag = (r[0]**2 + r[1]**2 + r[2]**2)**0.5
        # Acceleration
        Hern = self.G*M/(rmag * (ra + rmag)**2)*r
        
        return Hern
    
    
    # Define function to take into account disk acceleration using Miyamoto Nagai 1975 profile
    def MiyamotoNagaiAccel(self,M,r_d,r):
        """Inputs:
        M = mass of the disk
        r_d = scale length of disk
        r = relative position vector
        Returns:
        Miyamoto Nagai disk acceleration"""
        
        # Following the formula in the HW Instructions:
        z_d = self.rdisk/5.0
        R = (r[0]** r[1]**2)**0.5
        B = r_d + (r[2]**2 + z_d**2)**0.5
        # ZSTUFF is the terms associated with the z-direction
        ZSTUFF = 1/((r[2]**2 + z_d**2)**0.5)
        # This will allow for a different value for the z-component of the acceleration
        Arr = -self.G*M/(R**2 + B**2)**1.5*r*np.array([1,1,ZSTUFF])
        
       
        return Arr
     
    # Define a function that sums all acceleration vectors from each galaxy component
    def M31Accel(self,r):
        """Inputs:
        r = relative position vector
        Returns:
        Sum of each component of acceleration"""
        # Summing all the components together
        SUM = self.HernquistAccel(self.Mhalo, self.rhalo, r) + self.HernquistAccel(self.Mbulge, self.rbulge,r) + self.MiyamotoNagaiAccel(self.Mdisk, self.rdisk, r)

        return SUM
    
    
    # Define a function to integrate the acceleration vector over some amount of time
    def LeapFrog(self, dt, r, v):
        """Inputs:
        dt = small time interval
        r = position vector
        v = velocity vector
        Returns:
        New position and velocity vectors"""
        # Predicting the position at the next half timestep
        rhalf = r + v * dt/2
        # Predicting the final velocity at the next timestep using the acceleration field at the rhalf position
        vnew = v + self.M31Accel(rhalf)*dt
        # Approximation of the final position using the average expected speed over the time interval
        rnew = rhalf+0.5*vnew*dt
        
        return rnew, vnew 
    
    
    # Define a function to solve equations of motion and compute the future orbit of M33 for 10 Gyr in the future
    def OrbitIntegration(self, t0, dt, tmax):
        """Inputs:
        t0 = starting time
        dt = small time interval
        tmax = maximum time
        Returns:
        Array of positions and velocities saved to a file"""
        # Initialize the time to the input starting time
        t = t0
        # Initialize an empty array 
        orbit = np.zeros((int(tmax/dt)+2,7))
        
        # Initialize the first row of the orbit
        orbit[0] = t0, *tuple(self.r0), *tuple(self.v0)
        # Initialize a counter for the orbit starting at 1 or the next increment since we already calculated the 0th increment
        i = 1 
        # Start the integration (advancing in time steps and computing LeapFrog at each step)
        while (t < tmax): 
            
            # Advance the time by one timestep, dt
            t += dt
            # Store the new time in the first column of the ith row
            orbit[i,0] = t
            # Advance the position and velocity using the LeapFrog scheme
            rnew, vnew = self.LeapFrog(dt, orbit[i-1,1:4],orbit[i-1,4:7])
            # Store the new position vector into the columns with indices 1,2,3 of the ith row of orbit
            orbit[i,1:4] = rnew
            # Store the new velocity vector into the columns with indices 1,2,3 of the ith row of orbit
            orbit[i,4:7]=vnew
            
            # Update counter i, where i is keeping track of the number of rows
            i+=1
        
        
        # Writing the data to a file
        np.savetxt(self.filename, orbit, fmt = "%11.3f"*7, comments='#', 
                header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
        
        # there is no return function

In [12]:
"""Plotting the above code"""
# Create output file
AnaOrb = M33AnalyticOrbit("GalOrbit.txt")

AttributeError: 'numpy.ndarray' object has no attribute 'value'

In [61]:
"""Starting at 0 Gyr and ending at 10 Gyr with time increments of 0.5
Gyr and then 0.01 Gyr"""
AnaOrb.OrbitIntegration(0,0.01,10)

/home/mespn520/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in double_scalars


UnitConversionError: Can only apply 'add' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)